# Tools for Analyzing Time Series of Satellite Imagery (TATSSI)

## Download data from the GEE
TATSSI allows to dowload data products from Google Earth Engine [GEE](https://developers.google.com/earth-engine/datasets/) and create and process time series from different sensors such as:

* Moderate Resolution Imaging Spectroradiometer [MODIS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/#MODIS_anchor)
* Visible Infrared Imaging Radiometer Suite [VIIRS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/#S-NPPVIIRS_anchor)
* Sentinel-2 MultiSpectral Instrument [MSI](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)

Downloading products from the GEE requires a Google Account, please, first regsiter as a user to get the login credentials and follow the instructions to authenticate to the Earth Engine using the [Python API](https://developers.google.com/earth-engine/python_install).

1. Select your are of study, you can save it as a GeoJSON file, e.g. from the [geojson.io](http://geojson.io/) site.
2. For the time being you need to match the official MODIS product bands and the GEE bands since the band names are not consistent.
3. Generate the time series, you can then use the rest of the (awesome) TATSSI tools!

In [1]:
# Import libraries
import os
import json
from datetime import datetime as dt

import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

from helpers import utils
from TATSSI.download.gee_downloader import export_collection
from TATSSI.download.gee_downloader import get_variables

from TATSSI.time_series.generator import Generator

Enter verification code: 4/1AX4XfWhfe7B62Bx3HPuwlXx-VDNdCl6pwHvUaxEhfiSHix4-fQZP3Q1M3I4

Successfully saved authorization token.


In [2]:
# Area of study
with open('Leeds.geojson') as json_file:
    region = json.load(json_file)
    
region = region['features'][0]['geometry']['coordinates']
geometry = ee.Geometry.Polygon(region)
print(geometry)

ee.Geometry({
  "type": "Polygon",
  "coordinates": [
    [
      [
        -1.5763235092163086,
        53.77559550401067
      ],
      [
        -1.568073034286499,
        53.77559550401067
      ],
      [
        -1.568073034286499,
        53.77990661754899
      ],
      [
        -1.5763235092163086,
        53.77990661754899
      ],
      [
        -1.5763235092163086,
        53.77559550401067
      ]
    ]
  ],
  "evenOdd": true
})


For this example we will use the Sentinel-2 SR product.

In [3]:
# Dates as YYYY-MM-DD
start_date, end_date = '2021-01-01', '2021-12-31'

# Get Sentinel-2 MSI data
product_name = 'S2_SR'

# Product using GEE nomenclature
product = f'COPERNICUS/{product_name}'

# Get GEE collection
ee_collection = (ee.ImageCollection(product).
                 filterBounds(geometry).
                 filterDate(start_date, end_date).
                 select(['B1', 'B2', 'B3']))


In [4]:
# Change to corresponding UTM Zone
# e.g. for the UK 
crs = 'EPSG:32630'

task_b1 = ee.batch.Export.image.toCloudStorage(
    image=ee_collection.select('B1').toBands(),
    region=geometry,
    description='B1',
    bucket='clearsky',
    fileNamePrefix='B3_Leeds',
    scale=10,
    crs=crs,
    maxPixels=1e13)

task_b2 = ee.batch.Export.image.toCloudStorage(
    image=ee_collection.select('B2').toBands(),
    region=geometry,
    description='B2',
    bucket='clearsky',
    fileNamePrefix='B3_Leeds',
    scale=10,
    crs=crs,
    maxPixels=1e13)

task_b3 = ee.batch.Export.image.toCloudStorage(
    image=ee_collection.select('B3').toBands(),
    region=geometry,
    description='B3',
    bucket='clearsky',
    fileNamePrefix='B3_Leeds',
    scale=10,
    crs=crs,
    maxPixels=1e13)

In [5]:
task_b1.start()
task_b2.start()
task_b3.start()

INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/image:export?alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/image:export?alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/image:export?alt=json


In [17]:
task_b1.status()

INFO:googleapiclient.discovery:URL being requested: GET https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/operations/IR2ATOEQKUYXISL2GE4HWH32?alt=json


{'state': 'COMPLETED',
 'description': 'B1',
 'creation_timestamp_ms': 1650982701669,
 'update_timestamp_ms': 1650982867456,
 'start_timestamp_ms': 1650982792523,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://console.developers.google.com/storage/browser/clearsky/'],
 'id': 'IR2ATOEQKUYXISL2GE4HWH32',
 'name': 'projects/earthengine-legacy/operations/IR2ATOEQKUYXISL2GE4HWH32'}

In [18]:
task_b2.status()

INFO:googleapiclient.discovery:URL being requested: GET https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/operations/2CFB4R2PF4HXMNSICXRMSNX3?alt=json


{'state': 'RUNNING',
 'description': 'B2',
 'creation_timestamp_ms': 1650982702123,
 'update_timestamp_ms': 1650982958243,
 'start_timestamp_ms': 1650982955546,
 'task_type': 'EXPORT_IMAGE',
 'id': '2CFB4R2PF4HXMNSICXRMSNX3',
 'name': 'projects/earthengine-legacy/operations/2CFB4R2PF4HXMNSICXRMSNX3'}

In [19]:
task_b3.status()

INFO:googleapiclient.discovery:URL being requested: GET https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/operations/2I3GULTO2PS3LTZD3QYJPU4G?alt=json


{'state': 'READY',
 'description': 'B3',
 'creation_timestamp_ms': 1650982702682,
 'update_timestamp_ms': 1650982702682,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': '2I3GULTO2PS3LTZD3QYJPU4G',
 'name': 'projects/earthengine-legacy/operations/2I3GULTO2PS3LTZD3QYJPU4G'}

In [86]:
print(dt.fromtimestamp(task_b3.status()['start_timestamp_ms']/1000.0))
print(dt.fromtimestamp(task_b3.status()['update_timestamp_ms']/1000.0))

2021-08-31 15:04:28.571000
2021-08-31 15:10:56.450000


Get the data!

In [13]:
output_dir = '/tmp'
# Export image collection
datasets = export_collection(ee_collection, output_dir, bands=bands,
                             product=product, scale=None, crs='EPSG:32630',
                             region=region, file_per_band=True)

NameError: name 'bands' is not defined

In [ ]:
ee_object = ee.Image(ee_collection.toList( int(ee_collection.size().getInfo()) ).get(0))

In [ ]:
ee_collection.select?

In [ ]:
_date = ee_object.get('system:index').getInfo()
name = f'{_date}.tif'
filename = os.path.join(os.path.abspath(output_dir), name)

In [ ]:
params

In [ ]:
for band in bands:
    print(band, ee_object.select(band).projection().getInfo())

    
#url = ee_object.select(['B8A']).getDownloadURL(params)


Create TATSSI time series.

In [ ]:
# TATSSI Time Series Generator object
tsg = Generator(source_dir=output_dir, product=product_name,
                version=version, data_format='tif',
                progressBar=None, preprocessed=True)

# Create layerstacks
tsg._Generator__datasets = datasets
for dataset in tsg._Generator__datasets:
    tsg._Generator__generate_layerstack(dataset, 'tif')

# For the associated product layers, decode the corresponding QA bands
tsg._Generator__decode_qa('tif')